In this challenge, I applied Bag of Words and Tfidf to the 20 newsgroup dataset that is made available through the scikit learn dataset package.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import seaborn as sns
import re
import nltk
import string
import re
import sklearn
from sklearn import metrics    
import timeit
import spacy
import gensim

In [12]:
from sklearn.datasets import fetch_20newsgroups
dataset_full = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'), shuffle=True, random_state=42)
newsdf = pd.DataFrame()
newsdf['text'] = dataset_full.data
newsdf['source'] = dataset_full.target
label=[]
for i in newsdf['source']:
    label.append(dataset_full.target_names[i])
newsdf['label']=label

dataset_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), shuffle=True, random_state=42)
newsdf_train = pd.DataFrame()
newsdf_train['text'] = dataset_train.data
newsdf_train['source'] = dataset_train.target
label=[]
for i in newsdf_train['source']:
    label.append(dataset_train.target_names[i])
newsdf_train['label']=label

dataset_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), shuffle=True, random_state=42)
newsdf_test = pd.DataFrame()
newsdf_test['text'] = dataset_test.data
newsdf_test['source'] = dataset_test.target
label=[]
for i in newsdf_test['source']:
    label.append(dataset_test.target_names[i])
newsdf_test['label']=label

In [13]:
newsdf.head()

,text,source,label
0,\n\nI am sure some bashers of Pens fans are pr...,10,rec.sport.hockey
1,My brother is in the market for a high-perform...,3,comp.sys.ibm.pc.hardware
2,\n\n\n\n\tFinally you said what you dream abou...,17,talk.politics.mideast
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,3,comp.sys.ibm.pc.hardware
4,1) I have an old Jasmine drive which I cann...,4,comp.sys.mac.hardware


In [22]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics    
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import timeit
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from nltk.corpus import gutenberg, stopwords
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import normalize
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import fetch_20newsgroups
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [17]:
 import nltk
 nltk.download('stopwords')
stopWords = set(stopwords.words('english'))

def textcleaner_stem(text):
    ''' Takes in raw unformatted text and strips punctuation, removes whitespace,
    strips numbers, tokenizes and stems.
    Returns string of processed text to be used into CountVectorizer
    '''
    # Lowercase and strip everything except words
    cleaner = re.sub(r"[^a-zA-Z ]+", ' ', text.lower())
    # Tokenize
    cleaner = word_tokenize(cleaner)
    ps = PorterStemmer()
    clean = []
    for w in cleaner:
        # filter out stopwords
        if w not in stopWords:
            # filter out short words
            if len(w)>2:
                # Stem 
                clean.append(ps.stem(w))
    return ' '.join(clean)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [23]:
newsdf['clean_text_stem'] = newsdf.text.apply(lambda x: textcleaner_stem(x))
newsdf_train['clean_text_stem'] = newsdf_train.text.apply(lambda x: textcleaner_stem(x))
newsdf_test['clean_text_stem'] = newsdf_test.text.apply(lambda x: textcleaner_stem(x))

In [24]:
newsdf.head()

,text,source,label,clean_text_stem
0,\n\nI am sure some bashers of Pens fans are pr...,10,rec.sport.hockey,sure basher pen fan pretti confus lack kind po...
1,My brother is in the market for a high-perform...,3,comp.sys.ibm.pc.hardware,brother market high perform video card support...
2,\n\n\n\n\tFinally you said what you dream abou...,17,talk.politics.mideast,final said dream mediterranean new area greate...
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,3,comp.sys.ibm.pc.hardware,think scsi card dma transfer disk scsi card dm...
4,1) I have an old Jasmine drive which I cann...,4,comp.sys.mac.hardware,old jasmin drive use new system understand ups...


In [29]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')

stopWords = set(stopwords.words('english'))

def textcleaner_lemmas(text):
    ''' Takes in raw unformatted text and strips punctuation, removes whitespace,
    strips numbers, tokenizes and stems.
    Returns string of processed text to be used into CountVectorizer
    '''
    # Lowercase and strip everything except words
    cleaner = re.sub(r"[^a-zA-Z ]+", ' ', text.lower())
    # Tokenize
    cleaner = word_tokenize(cleaner)
    ps = PorterStemmer()
    clean = []
    for w in cleaner:
        # filter out stopwords
        if w not in stopWords:
            # filter out short words
            if len(w)>2:
                # lemmatizer 
                clean.append(lemmatizer.lemmatize(w))
    return ' '.join(clean)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [30]:
newsdf['clean_text_lemma'] = newsdf.text.apply(lambda x: textcleaner_lemmas(x))
newsdf_train['clean_text_lemma'] = newsdf_train.text.apply(lambda x: textcleaner_lemmas(x))
newsdf_test['clean_text_lemma'] = newsdf_test.text.apply(lambda x: textcleaner_lemmas(x))

In [32]:
from sklearn.feature_extraction.text import CountVectorizer

BOWvectorizer = CountVectorizer(analyzer='word')

x_train_stem = BOWvectorizer.fit_transform(newsdf_train['clean_text_stem'])
y_train_stem = newsdf_train['source']
x_test_stem = BOWvectorizer.transform(newsdf_test['clean_text_stem'])
y_test_stem = newsdf_test['source']
features_train = BOWvectorizer.get_feature_names()
len(features_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


55161

In [35]:
BOWvectorizer = CountVectorizer(analyzer='word')

x_train_lemma = BOWvectorizer.fit_transform(newsdf_train['clean_text_lemma'])
y_train_lemma = newsdf_train['source']
x_test_lemma = BOWvectorizer.transform(newsdf_test['clean_text_lemma'])
y_test_lemma = newsdf_test['source']
features_train = BOWvectorizer.get_feature_names()
len(features_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


66613

In [38]:
X_train =  x_train_lemma
y_train = y_train_lemma
X_test = x_test_lemma 
y_test = y_test_lemma

In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# Models
lr = LogisticRegression()
rfc = RandomForestClassifier()
gbc = GradientBoostingClassifier()

lr.fit(X_train, y_train)
rfc.fit(X_train, y_train)
gbc.fit(X_train, y_train)

print("----------------------Logistic Regression Scores----------------------")
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

print("----------------------Random Forest Scores----------------------")
print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

print("----------------------Gradient Boosting Scores----------------------")
print('Training set score:', gbc.score(X_train, y_train))
print('\nTest set score:', gbc.score(X_test, y_test))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


----------------------Logistic Regression Scores----------------------
Training set score: 0.9700371221495492

Test set score: 0.6141795007966012
----------------------Random Forest Scores----------------------
Training set score: 0.9737493371044723

Test set score: 0.6123207647371216
----------------------Gradient Boosting Scores----------------------
Training set score: 0.8604383949089623

Test set score: 0.6066117896972916


In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer

Tfidfvectorizer = TfidfVectorizer(
    max_df=0.5, min_df=2, use_idf=True, norm=u'l2', smooth_idf=True)

x_train_tfidf = Tfidfvectorizer.fit_transform(newsdf_train['clean_text_lemma'])
y_train_tfidf = newsdf_train['source']
x_test_tfidf = Tfidfvectorizer.transform(newsdf_test['clean_text_lemma'])
y_test_tfidf = newsdf_test['source']
features_train = Tfidfvectorizer.get_feature_names()
len(features_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


29942

In [41]:
X_train =  x_train_tfidf
Y_train = y_train_tfidf
X_test = x_test_tfidf 
Y_test = y_test_tfidf

In [42]:
lrtfidf = LogisticRegression()
rfctfidf = RandomForestClassifier()
gbctfidf = GradientBoostingClassifier()

lrtfidf.fit(X_train, y_train)
rfctfidf.fit(X_train, y_train)
gbctfidf.fit(X_train, y_train)

print("----------------------Logistic Regression Scores----------------------")
print('Training set score:', lrtfidf.score(X_train, y_train))
print('\nTest set score:', lrtfidf.score(X_test, y_test))

print("----------------------Random Forest Scores----------------------")
print('Training set score:', rfctfidf.score(X_train, y_train))
print('\nTest set score:', rfctfidf.score(X_test, y_test))

print("----------------------Gradient Boosting Scores----------------------")
print('Training set score:', gbctfidf.score(X_train, y_train))
print('\nTest set score:', gbctfidf.score(X_test, y_test))

----------------------Logistic Regression Scores----------------------
Training set score: 0.9025985504684462

Test set score: 0.6833510355815189
----------------------Random Forest Scores----------------------
Training set score: 0.9733074067526958

Test set score: 0.611125862984599
----------------------Gradient Boosting Scores----------------------
Training set score: 0.9117023157150433

Test set score: 0.5950610727562401


Neither one of the models seemed to predict the news topic very well, but it looks like the Tfidf model had training and test scores that were a little closer together so it may have reduced the overfitting problem.

I am not exactly sure why the models didn't work as well. In the program, we focused on distinguishing between two authors. Maybe it's possible that since there are 20 different news topics in this data set, the multiple classification task is too much for the models I generated.